In [3]:
import os
import subprocess
import sagemaker
import tensorflow as tf
from sagemaker import get_execution_role


sagemaker_session = sagemaker.Session()

_role = get_execution_role()

In [ ]:
inputs="s3://sagemaker-us-east-1-002224604296/denoise/tfrecords/"

In [ ]:
# !pygmentize 'source/core/step_three_recognize_process/tools/train/

#single gpu training
from sagemaker.tensorflow import TensorFlow

# Canbe local_gpu, local, or ml.p3.2xlarge, ml.p3.8xlarge etc.
instance_type='ml.p3.2xlarge'
bucket= "sagemaker-us-east-1-002224604296"
prefix="train-0611"
hyperparameters = {'servable_model_dir': '/opt/ml/model', 'epoch': 1, 'task_type': 'train', 'sagemaker': 'true'
                  }
# requirements.txt are not allowed in script mode (BYOS). https://stackoverflow.com/q/53530867/3252127
# Hyperparameters are specified in the model_config.py file, we need to extract them out to use SageMaker automl features.
segan_estimator = TensorFlow(entry_point='source/main_train_deploy.py',
                             source_dir='.',
                             model_dir='/opt/ml/model',
                             role=_role,
                             output_path='s3://{}/{}/output'.format(bucket, prefix),
                             framework_version='1.15.2',
                             hyperparameters=hyperparameters, #set weights_path if you are finetune a pretrained model
                             py_version='py2',
                             train_instance_type=instance_type,
                             train_instance_count=1)

In [ ]:
result = segan_estimator.fit(inputs)

In [ ]:
predictor=segan_estimator.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge', endpoint_type='tensorflow-serving')

In [ ]:
from scipy.io import wavfile
import tensorflow as tf
import numpy as np
import time

def de_emph(y, coeff=0.95):
    if coeff <= 0:
        return y
    x = np.zeros(y.shape[0], dtype=np.float32)
    x[0] = y[0]
    for n in range(1, y.shape[0], 1):
        x[n] = coeff * x[n - 1] + y[n]
    return x

def pre_emph_np(x, coeff=0.95):
    x0 = np.reshape(x[0], [1,])
    diff = x[1:] - coeff * x[:-1]
#     concat = tf.concat(0, [x0, diff])
    concat = np.concatenate([x0, diff], 0)
    return concat

def pre_emph_test_np(coeff, wav):
    x_preemph = pre_emph_np(wav, coeff)
    return x_preemph
    
def clean_serving(x, canvas_size, preemph, predictor=None):
    """ clean a utterance x
        x: numpy array containing the normalized noisy waveform
    """
    c_res = None
    print('start timer')
    Time1 = time.time()
    for beg_i in range(0, x.shape[0], canvas_size):
        if x.shape[0] - beg_i  < canvas_size:
            length = x.shape[0] - beg_i
            pad = canvas_size - length
        else:
            length = canvas_size
            pad = 0
        x_ = np.zeros((1, canvas_size))
        print(x_[0].shape)
        if pad > 0:
            x_ = np.concatenate((x[beg_i:beg_i + length], np.zeros(pad)))
        else:
            x_ = x[beg_i:beg_i + length]
        print('Cleaning chunk {} -> {}'.format(beg_i, beg_i + length))
        canvas_w = None
        # fdict = {self.gtruth_noisy[0]:x_}
        # canvas_w = self.sess.run(self.Gs[0],
        #                         feed_dict=fdict)[0]
        test_example = {'wav_and_noisy': x_.tolist()}
        if predictor == None:
            canvas_w = x_
        else:
            canvas_w = predictor.predict(test_example)
        canvas_w = np.reshape(np.array(canvas_w['predictions']),canvas_size)
        print('canvas w shape: ', canvas_w.shape)
        if pad > 0:
            print('Removing padding of {} samples'.format(pad))
            # get rid of last padded samples
            canvas_w = canvas_w[:-pad]
        if c_res is None:
            c_res = canvas_w
        else:
            c_res = np.concatenate((c_res, canvas_w))
    # deemphasize
    c_res = de_emph(c_res, preemph)
    print('finish {}'.format(time.time()-Time1))
    return c_res

In [ ]:
test_wav = "../../record_noise/record_noise_28_clip_0.wav"
preemph = 0.95
canvas_size = 2**14

fm, wav_data = wavfile.read(test_wav)
wavname = test_wav.split('/')[-1]
wave = (2./65535.) * (wav_data.astype(np.float32) - 32767) + 1.

x_pholder_np = pre_emph_test_np(preemph, wave)

c_wave = clean_serving(x=x_pholder_np, canvas_size=canvas_size, preemph=preemph, predictor=predictor)

wavfile.write(os.path.join('../../clean', wavname), 16e3, c_wave)